In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
import pandas as pd
import librosa
import numpy as np
from sklearn.pipeline import make_pipeline

In [24]:
#load data
train = torchaudio.datasets.LIBRITTS('./', url="train-clean-100", download=True)
test = torchaudio.datasets.LIBRITTS('./', url="test-clean", download=True)

In [26]:
speakers = pd.read_csv("/content/LibriTTS/speakers.tsv", sep='\t')
speakers = speakers[['READER', 'GENDER']]
speakers = speakers.to_dict().get('READER')

In [27]:
#prepare data
import random
order = [i for i in range(len(train))]
random.shuffle(order)
X_train = [np.mean(librosa.feature.mfcc(np.array(train[i][0][0]), train[i][1]), axis=1) for i in order]
y_train = [0 if speakers.get(train[i][4]) == 'F' else 1 for i in order]
X_test = [np.mean(librosa.feature.mfcc(np.array(test[i][0][0]), test[i][1]), axis=1) for i in range(len(test))]
y_test = [0 if speakers.get(test[i][4]) == 'F' else 1 for i in range(len(test))]


In [28]:
#RBF SVM
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
model_svm = make_pipeline(StandardScaler(), SVC())
model_svm.fit(X_train, y_train)
y_pred = model_svm.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score
svm_accuracy = accuracy_score(y_test, y_pred)
print("RBF SVM Accuracy = ", svm_accuracy)

RBF SVM Accuracy =  0.8780235683274757


In [30]:
#Linear SVM
from sklearn.svm import LinearSVC
model_svm_linear = make_pipeline(StandardScaler(), LinearSVC(max_iter=5000))
model_svm_linear.fit(X_train, y_train)
y_pred_linear = model_svm_linear.predict(X_test)
svm_linear_accuracy = accuracy_score(y_test, y_pred_linear)
print("Linear SVM Accuracy =", svm_linear_accuracy)

Linear SVM Accuracy = 0.9162704155468265
